**Q1. Create a simple Python script that starts two threads; each thread should print numbers from 1 to 5 with a delay of 1 second between each number.**

In [ ]:
from time import sleep
from threading import Thread

DELAY = 1
NO_OF_THREADS = 2

def task(thread_no):
  for i in range(1, 6):
    print(f"Thread {thread_no}: {i}")
    sleep(DELAY)

if __name__ == '__main__':
  threads = []
  for thread_no in range(NO_OF_THREADS):
    thread = Thread(target=task, args=(thread_no+1,))
    threads.append(thread)
    thread.start()

  for thread in threads:
    thread.join()

Thread 1: 1
Thread 2: 1
Thread 1: 2
Thread 2: 2
Thread 1: 3
Thread 2: 3
Thread 1: 4
Thread 2: 4
Thread 1: 5
Thread 2: 5


**Q2. Write a Python program where two threads print alternating messages like "Ping" and "Pong" respectively.**

In [ ]:
from time import sleep
from threading import Thread, Event

PING_SLEEP = 1
PONG_SLEEP = 2
NO_OF_TURNS = 6

class PingPongGame:
  def __init__(self, no_of_turns):
    self.no_of_turns = no_of_turns
    self.ping_event = Event()
    self.pong_event = Event()
    self.ping_event.set()  # Start with Ping

  def ping(self):
    for _ in range(self.no_of_turns):
      self.ping_event.wait()
      print("Ping")
      sleep(PING_SLEEP)
      self.pong_event.set()
      self.ping_event.clear()

  def pong(self):
    for _ in range(self.no_of_turns):
      self.pong_event.wait()
      print("Pong")
      sleep(PONG_SLEEP)
      self.ping_event.set()
      self.pong_event.clear()

if __name__ == "__main__":
  ping_pong = PingPongGame(NO_OF_TURNS)
  ping_thread = Thread(target=ping_pong.ping)
  pong_thread = Thread(target=ping_pong.pong)
  ping_thread.start()
  pong_thread.start()

  ping_thread.join()
  pong_thread.join()

Ping
Pong
Ping
Pong
Ping
Pong
Ping
Pong
Ping
Pong
Ping
Pong


**Q3. Write a Python program using threads where a semaphore is used to limit access to a shared resource. For example, simulate a scenario where only a certain number of threads can write to a shared file at the same time. But feel free to choose any setting.**

P.S. You can just copy-paste from the colab we used in the Session. But make sure you understand it!

In [ ]:
# Scenario of allowing only two persons on one bike at a given time such that every person can use the bike.
from time import sleep
from threading import Thread, Semaphore

BIKE_USAGE_TIME = 3
AVAILABLE_MEMBERS = 10
PERSONS_ALLOWED_ON_BIKE = 2

class Bike:
  def __init__(self, allowed_persons):
    self._semaphore = Semaphore(allowed_persons)
    self.bike_users = 0

  def on_bike(self, person_id):
    with self._semaphore:
      print(f"PERSON {person_id} is on the bike.")
      self.bike_users += 1
      print(f"No. of Persons using the bike = {self.bike_users}")
      sleep(BIKE_USAGE_TIME)
      print(f"PERSON {person_id} got off the bike.")
      self.bike_users -= 1


if __name__ == '__main__':
  persons = []
  bike = Bike(PERSONS_ALLOWED_ON_BIKE)
  for person_id in range(AVAILABLE_MEMBERS):
    person = Thread(target=bike.on_bike, args=(person_id+1,))
    persons.append(person)
    person.start()

  for person in persons:
    person.join()

  print(f"No. of Persons using the bike = {bike.bike_users}")
  print("All available persons have used the bike.")

PERSON 1 is on the bike.
No. of Persons using the bike = 1
PERSON 2 is on the bike.
No. of Persons using the bike = 2
PERSON 1 got off the bike.
PERSON 3 is on the bike.
No. of Persons using the bike = 2
PERSON 2 got off the bike.
PERSON 4 is on the bike.
No. of Persons using the bike = 2
PERSON 3 got off the bike.
PERSON 5 is on the bike.
No. of Persons using the bike = 2
PERSON 4 got off the bike.
PERSON 6 is on the bike.
No. of Persons using the bike = 2
PERSON 5 got off the bike.
PERSON 7 is on the bike.
No. of Persons using the bike = 2
PERSON 6 got off the bike.
PERSON 8 is on the bike.
No. of Persons using the bike = 2
PERSON 7 got off the bike.PERSON 8 got off the bike.
PERSON 9 is on the bike.
No. of Persons using the bike = 2

PERSON 10 is on the bike.
No. of Persons using the bike = 2
PERSON 9 got off the bike.
PERSON 10 got off the bike.
No. of Persons using the bike = 0
All available persons have used the bike.


**Q4. [OPTIONAL] Write a simple usecase of Mutex in C++. No need to run just dump the code in multiline comment below.**

In [ ]:
"""
#include <iostream>
#include <thread>
#include <mutex>

std::mutex mtx; // Mutex for protecting the counter
int counter = 0; // Shared counter variable

void incrementCounter() {
    for (int i = 0; i < 1000000; ++i) {
        // Acquire the lock
        mtx.lock();

        // Critical section: Increment the counter
        ++counter;

        // Release the lock
        mtx.unlock();
    }
}

int main() {
    std::thread thread1(incrementCounter);
    std::thread thread2(incrementCounter);

    thread1.join();
    thread2.join();

    std::cout << "Final counter value: " << counter << std::endl;

    return 0;
}
"""

**Q5. [OPTIONAL] DINING PHILOSOPHER'S PROBLEM**

* There are five philosophers sitting around a dining table.
* Each philosopher thinks and eats spaghetti.
* To eat, a philosopher must have two forks (one on the left and one on the right).
* There is a single fork between each pair of adjacent philosophers.
* Philosophers must pick up both forks to eat and put them down when they're done.

In [ ]:
import time
import threading
from threading import Lock

THINKING_TIME = 5
EATING_TIME = 1
MAX_SERVINGS = 3
NO_OF_PHILOSOPHERS = 5

class Philosopher(threading.Thread):
    def __init__(self, name, left_fork, right_fork):
        threading.Thread.__init__(self)
        self.name = name
        self.left_fork = left_fork
        self.right_fork = right_fork
        self.serving_count = 0
        self.max_servings = MAX_SERVINGS

    def run(self):
        while self.serving_count < self.max_servings:
            # Think
            print(f"{self.name} is thinking")
            time.sleep(THINKING_TIME)

            # Acquire forks (in sorted order to avoid deadlock)
            if id(self.left_fork) < id(self.right_fork):
                self.left_fork.acquire()
                self.right_fork.acquire()
            else:
                self.right_fork.acquire()
                self.left_fork.acquire()

            # Eat
            print(f"{self.name} is eating")
            time.sleep(EATING_TIME)
            self.serving_count += 1

            # Release forks
            if id(self.left_fork) < id(self.right_fork):
                self.left_fork.release()
                self.right_fork.release()
            else:
                self.right_fork.release()
                self.left_fork.release()

        print(f"{self.name} has finished eating")

if __name__ == '__main__':
  forks = [Lock() for _ in range(NO_OF_PHILOSOPHERS)]

  philosophers = []
  for i in range(NO_OF_PHILOSOPHERS):
      left_fork = forks[i]
      right_fork = forks[(i + 1) % 5]
      philosopher = Philosopher(f"Philosopher {i + 1}", left_fork, right_fork)
      philosophers.append(philosopher)
      philosopher.start()

  for philosopher in philosophers:
      philosopher.join()

  print("All philosophers have finished eating")

Philosopher 1 is thinking
Philosopher 2 is thinking
Philosopher 3 is thinking
Philosopher 4 is thinking
Philosopher 5 is thinking
Philosopher 1 is eating
Philosopher 4 is eating
Philosopher 4 is thinking
Philosopher 1 is thinking
Philosopher 5 is eating
Philosopher 2 is eating
Philosopher 2 is thinking
Philosopher 3 is eating
Philosopher 5 is thinking
Philosopher 3 is thinking
Philosopher 4 is eating
Philosopher 1 is eating
Philosopher 4 is thinkingPhilosopher 1 is thinkingPhilosopher 2 is eating


Philosopher 5 is eating
Philosopher 2 is thinking
Philosopher 5 is thinking
Philosopher 3 is eating
Philosopher 3 is thinking
Philosopher 1 is eating
Philosopher 4 is eating
Philosopher 1 has finished eating
Philosopher 4 has finished eatingPhilosopher 5 is eating

Philosopher 2 is eating
Philosopher 5 has finished eating
Philosopher 2 has finished eating
Philosopher 3 is eating
Philosopher 3 has finished eating
All philosophers have finished eating
